In [1]:
###################################################################################################
# Define parameters and query data
###################################################################################################

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata

variable = 'snow' # Snow (Daily Snowfall)

# Location and bounding box
location_name = 'St. Louis, MO'
nlat = 38.9     # northern latitude
slat = 38.5     # southern latitude
wlon = -90.55   # western longitude
elon = -90.17   # eastern longitude

# Query variable
var, meta = stndata.bbox_multistn_daily(elem=variable,slat=slat,nlat=nlat,wlon=wlon,elon=elon,
                                        print_md=False)

snow: Reading in 57 total stations (station id: name, state) ...
#1. 112614: EAST ST LOUIS PARKS COLLEGE, IL
#2. 111160: CAHOKIA, IL
#3. 238791: WEBSTER GROVES, MO
#4. 238561: VALLEY PARK, MO
#5. 237452: ST LOUIS SCIENCE CENTER, MO
#6. 93963: ST LOUIS EADS BRIDGE, MO
#7. 237465: ST LOUIS ST LOUIS UNIV, MO
#8. 237470: SAINT LOUIS WASHINGTON UNIV, MO
#9. 238525: UNIVERSITY CITY, MO
#10. US1MOSLC007: ST. LOUIS 5.7 SW, MO
#11. 234272: JEFFERSON BARRACKS, MO
#12. US1MOSL0003: WEBSTER GROVES 0.9 ESE, MO
#13. US1MOSL0004: KIRKWOOD 1.6 S, MO
#14. US1MOSL0018: MANCHESTER 1.4 SE, MO
#15. US1MOSL0020: LADUE 1.6 N, MO
#16. US1MOSLC004: ST. LOUIS 1.5 S, MO
#17. US1MOSLC005: ST. LOUIS 2.4 S, MO
#18. US1MOSL0029: AFFTON 0.8 WNW, MO
#19. US1MOSL0035: ST. LOUIS 1.0 SW, MO
#20. US1MOSLC006: ST. LOUIS 4.9 SW, MO
#21. US1MOSL0045: CRESTWOOD 0.4 NW, MO
#22. US1MOSL0048: ST. LOUIS 6.3 SW (CLOSED), MO
#23. US1MOSL0049: BALLWIN 1.6 E, MO
#24. US1MOSL0050: WEBSTER GROVES 1.6 NNE, MO
#25. US1MOSL0054: WEBSTER G

In [2]:
# PLOT

# Use this to read in packages from other directories ---------
import sys, os 
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.processing.bbox_my import bbox_avg_my, bbox_max_my, bbox_min_my
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
import warnings

def timeseries_snow_plot(var,meta,location_name,nlat,slat,wlon,elon,month1,month2,num_days,num_mons,num_stns,
                         method,incl_tl,tl_syr,tl_eyr,minbuff,maxbuff,ymajtick,ymintick,xmajtick,xmintick,
                         incl_map,img_tile,lbl_buff,ext_buff):
        
    ############################################################################################################# 
    # Convert months string into list of months as integers
    #############################################################################################################

    # Lists of month strings
    month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    mon1 = month_names[month_names.index(month1):]      # all elements to the right (inclusive) of month1
    mon2 = month_names[:(month_names.index(month2)+1)]  # all elements to the left (inclusive) of month 2

    # Combine lists of months to create single list 
    if [i for i in mon1 if i in mon2] != []:  # there is overlap between lists
        mon_str = [i for i in mon1 if i in mon2]
    else:                                     # there is no overlap between lists
        mon_str = mon1 + mon2

    # Convert strings to integers
    monthi = [month_names.index(i)+1 for i in mon_str]

    ############################################################################################################# 
    # Create time series variable and apply data quality filters 
    #############################################################################################################

    # Mask var based on data quality standards (num_days, num_mons, num_stns)
    var_filt = var.copy()

    # Add 'Year' and 'Month' to filter
    if 'Year' not in var_filt.columns and 'Month' not in var_filt.columns:
        var_filt.insert(0,'Year',var_filt['Date'].dt.year),var_filt.insert(1,'Month',var_filt['Date'].dt.month)

    months_that_pass = var_filt.groupby(['Year','Month']).apply(lambda x: x.iloc[:,3:].count()) >= num_days
    years_that_pass = months_that_pass.groupby(level='Year').sum() >= num_mons
    stns_that_pass = years_that_pass.sum(axis=1) >= num_stns

    # Apply data quality standards to year's average value
    var_mask = var.copy()
    var_mask.loc[var_mask['Date'].dt.year.isin(list(stns_that_pass[stns_that_pass == False].index)),:] = np.nan
    var_mask['Date'] = var['Date']

    # dataframe is masked by this line, next lines are to decide which method to use

    # Determine which method to plot: rx1day methods
    if method == 'rx1day-max':
        var_my = bbox_max_my(var_mask)
        ts_pre = var_my.iloc[:,1:].loc[var_my['Month'].isin(monthi)].max()
    if method == 'rx1day-mean':
        # This way works but is slow - try to speed it up
        print('This method may take a few minutes...')
        warnings.filterwarnings('ignore',category=RuntimeWarning)
        ts_pre_col = pd.DataFrame({'Year':var['Date'].dt.year.unique()}) 
        for c in var_mask.columns[1:]:
            var_my_by_col = bbox_max_my(pd.DataFrame({'Date':var_mask['Date'],c:var_mask[c]}))
            ts_pre_col[c] = np.nanmax(var_my_by_col.iloc[:,1:].loc[var_my_by_col['Month'].isin(monthi)],axis=0)
        ts_pre = pd.Series(np.nanmean(ts_pre_col.iloc[:,1:],axis=1),index=np.array(ts_pre_col['Year']).flatten())

    # Determine which method to plot: region mean methods
    if method == 'alldays-mean':  # first part
        var_my = bbox_avg_my(var_mask)
    if method == 'snowdays-mean': # first part
        var_mask_snowdays = var_mask[[col for col in var_mask.columns if col != 'Date']].applymap(
                             lambda x: x if x > 0.00001 else np.nan) # set 0.'s and trace vals to NaN
        var_mask_snowdays['Date'] = var_mask['Date'] # add 'Date' back in for bbox_avg_my
        var_my = bbox_avg_my(var_mask_snowdays)
        
    if method == 'alldays-mean' or method == 'snowdays-mean': # second part
        mnly_vals = var_my.iloc[:,1:].loc[var_my['Month'].isin(monthi)]
        mnly_wgts = np.array([0.08493151,0.076712325,0.08493151,0.08219178,0.08493151,0.08219178,
                              0.08493151,0.08493151, 0.08219178,0.08493151,0.08219178,0.08493151])
        wgts = mnly_wgts if len(monthi) == 12 else mnly_wgts[[m-1 for m in monthi]]/np.sum(
                                                                           mnly_wgts[[m-1 for m in monthi]])
        ts_pre = pd.Series(index=mnly_vals.columns,dtype='float')
        for i, c in enumerate(mnly_vals.columns):
            if mnly_vals[c].isna().any(): # if any of the monthly values is NaN
               ts_pre.iloc[i] = np.nan
            else:
               ts_pre.iloc[i] = (mnly_vals[c]*wgts).sum() / wgts.sum()
            
    # Remove NaN values
    ts_pre = ts_pre.loc[str(ts_pre.first_valid_index()):str(ts_pre.last_valid_index())]
    ts = pd.DataFrame({'Year':np.array(ts_pre.index,dtype=int),'Value':ts_pre.reset_index(drop=True)})
    
    ############################################################################################################# 
    # Define figure 
    ############################################################################################################# 

    fig, ax = plt.subplots(figsize=[8,4],dpi=300)

    # Define x-axis years
    try:
       xtime = pd.Series(pd.date_range(start=str(ts['Year'].iloc[0])+'-01-01',
                                         end=str(ts['Year'].iloc[-1])+'-01-01',freq='YS'))
    except IndexError:
       print('ERROR: No data exists at this high a data quality standard. Reduce one of the options for '+
             ' data quality')

    ############################################################################################################# 
    # PLOT TIME SERIES 
    ############################################################################################################# 

    ax.plot(xtime,ts['Value'],'o-',markersize=0.5,c='darkcyan',lw=1)

    if incl_tl == True:

        # Determine years for trendline
        tl_syr = xtime.dt.year[0] if tl_syr == 'Start' else tl_syr
        tl_eyr = xtime.iloc[-1].year if tl_eyr == 'End' else tl_eyr

        # Extract subset of variables based on tl_syr/tl_eyr
        xtime_tl = xtime.loc[xtime.between(pd.to_datetime(str(tl_syr)+'-01-01'),
                                           pd.to_datetime(str(tl_eyr)+'-01-01'))].reset_index(drop=True)
        ts_tl = ts['Value'].loc[ts['Year'].between(int(tl_syr),int(tl_eyr))].reset_index(drop=True)

        # Account for NaNs when calculating polyfit
        ifinite = np.isfinite(np.array(xtime_tl)) & np.isfinite(np.array(ts_tl))
        m,b = np.polyfit(np.arange(len(xtime_tl))[ifinite],np.array(ts_tl)[ifinite],1)
        trendline = pd.Series(m * np.arange(len(xtime_tl)) + b)

        # Plot trendline and display slope as text on figure
        ax.plot(xtime_tl,trendline,'-',c='k',lw=0.5)
        sign = '+' if m > 0 else ''
        ax.text(xtime_tl.iloc[-1]+pd.Timedelta(days=365),trendline.iloc[-1],sign+str(round(m,4))+' in/yr',
                fontsize=5,bbox=dict(boxstyle='square',pad=0.1,edgecolor='none',facecolor='w'))

    ############################################################################################################# 
    # Features to include? Can toggle on and off in parameters
    ############################################################################################################# 

    if incl_map != 'False':
        bbox_to_anchor = (0,0,1,0.94) if incl_map == 'True (right)' else (0,0,0.15,0.94)
        inset_map(ax=ax,meta=meta,var=var,width=0.7,height=0.8,markercolor='k',slat=slat,nlat=nlat,
                  wlon=wlon,elon=elon,bbox_to_anchor=bbox_to_anchor,lbl_buff=lbl_buff,
                  proj=cartopy.crs.PlateCarree(),ext_buff=ext_buff,img_tile=img_tile)

    ############################################################################################################# 
    # Title and text
    ############################################################################################################# 

    ax.text(1,0.015,r'$\bf{DATA:}$ NOAA ACIS (http://data.rcc-acis.org) '+
                   r' $\bf{IMAGE:}$ Alex Thompson (@ajtclimate)',ha='right',fontsize=5,transform=ax.transAxes)
    season = f'{month1}-{month2}' if len(monthi) < 12 and month1 != month2 else (
                                                                        month1 if month1 == month2 else 'Annual')
    ax.set_title(season+' Daily Snowfall in '+location_name,loc='center',fontsize=14,pad=5,weight='bold');
    if method == 'rx1day-max':
       method_text = "Maximum individual station's Rx1day within the bounding box"
    if method == 'rx1day-mean':
       method_text = "Mean Rx1day for all stations within the bounding box"
    if method == 'alldays-mean':
       method_text = "Mean rainfall for all days from stations within bounding box"
    if method == 'raindays-mean':
       method_text = "Mean rainfall for rain days (>0) from stations within bounding box"
    ax.text(0.5,0.95,method_text,fontsize=9,ha='center',transform=ax.transAxes)

    ############################################################################################################# 
    # Axes specs 
    ############################################################################################################# 

    ax.set_ylabel('inches')
    ax.get_xaxis().set_major_locator(mpl.dates.YearLocator(base=xmajtick))
    ax.get_xaxis().set_minor_locator(mpl.dates.YearLocator(base=xmintick))
    ax.get_yaxis().set_major_locator(mpl.ticker.MultipleLocator(ymajtick))
    ax.get_yaxis().set_minor_locator(mpl.ticker.MultipleLocator(ymintick))
    ax.grid(alpha=0.05)
    ax.set_ylim([np.nanmin(ts['Value'])-minbuff,np.nanmax(ts['Value'])+maxbuff]);
    ax.spines[['right','top']].set_visible(False)

    return fig, ts


In [3]:
# WIDGET

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
from ClimateDataVisualizer.interactives import plots
from ClimateDataVisualizer.downloads.file_options import pdf_opts, xcl_opts         
import ipywidgets as ipyw
from IPython.display import display, HTML

# URLs to use for user help guides, needs double quotes
url_annualcycle = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-annual-cycle"
url_timeseries  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-time-series"
url_cumulative  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-cumulative"
url_map         = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-map-of-stations"
url_yaxis       = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-y-axis"

def timeseries_snow_widget(var,meta,location_name,nlat,slat,wlon,elon):

    #----------------------------------------------------------------------------------------------
    # Set up parameters to toggle
    #----------------------------------------------------------------------------------------------

    # Specify months to include
    txt_monthi = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_timeseries} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Months to include</a></h3>')
    text_mon1 = ipyw.Label(value='From',layout=ipyw.Layout(width='50px'))
    month1    = ipyw.Dropdown(options=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct',
                                       'Nov','Dec'],value='Jan',layout=ipyw.Layout(width='80px'))
    text_mon2 = ipyw.Label(value='To',layout=ipyw.Layout(width='50px'))
    month2    = ipyw.Dropdown(options=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct',
                                       'Nov','Dec'],value='Dec',layout=ipyw.Layout(width='80px'))

    # Method and plot features
    txt_meth = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_timeseries} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Choose what to plot</a></h3>')
    text_meth = ipyw.Label(value='Method to use',layout=ipyw.Layout(width='100px'))
    method = ipyw.Dropdown(options=[('Max Rx1day','rx1day-max'), 
                                   #('Mean Rx1day','rx1day-mean'), # removing this option for now
                                    ('Mean of all days','alldays-mean'),
                                    ('Mean of snow days','snowdays-mean')],value='rx1day-max',
                           layout=ipyw.Layout(width='125px'))
    text_tl = ipyw.Label(value='Include trendline?',layout=ipyw.Layout(width='120px'))
    incl_tl = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='80px'))
    text_yr = ipyw.Label(value='Trendline Years',layout=ipyw.Layout(width='100px'))
    tl_syr = ipyw.Dropdown(options=['Start',*np.sort(var['Date'].dt.year.unique())[:-1]],
                           value='Start',layout=ipyw.Layout(width='60px'))
    tl_eyr = ipyw.Dropdown(options=['End',*np.sort(var['Date'].dt.year.unique())[::-1][1:]],
                           value='End',layout=ipyw.Layout(width='60px'))

    # Data quality standards
    txt_dq = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_timeseries} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Data quality</a></h3>')
    text_ndays = ipyw.Label(value='Number of days',layout=ipyw.Layout(width='120px'))
    num_days = ipyw.Dropdown(options=np.arange(28),value=15,layout=ipyw.Layout(width='60px'))
    text_nmons = ipyw.Label(value='Number of months',layout=ipyw.Layout(width='120px'))
    num_mons = ipyw.Dropdown(options=np.arange(13),value=12,layout=ipyw.Layout(width='60px'))
    text_stns = ipyw.Label(value='Number of stations',layout=ipyw.Layout(width='120px'))
    if len(meta) == 1:
       num_stns = ipyw.Dropdown(options=[1],value=1,layout=ipyw.Layout(width='60px'))
    else:
       num_stns = ipyw.Dropdown(options=np.arange(1,len(meta)),value=1,
                                layout=ipyw.Layout(width='60px'))

    # Tick stride
    txt_tick = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis tick stride</a></h3>')
    ymajtk_desc = ipyw.Label(value='Major Y tick stride',layout=ipyw.Layout(width='120px'))
    ymajtick    = ipyw.FloatText(value=1.,layout=ipyw.Layout(width='60px'))
    
    ymajtk_unit = ipyw.Label(value='inches')
    ymintk_desc = ipyw.Label(value='Minor Y tick stride',layout=ipyw.Layout(width='120px'))
    ymintick    = ipyw.FloatText(value=0.5,layout=ipyw.Layout(width='60px'))
    ymintk_unit = ipyw.Label(value='inches')
    xmajtk_desc = ipyw.Label(value='Major X tick stride',layout=ipyw.Layout(width='120px'))
    xmajtick    = ipyw.IntText(value=20,layout=ipyw.Layout(width='60px'))
    xmajtk_unit = ipyw.Label(value='years')
    xmintk_desc = ipyw.Label(value='Minor X tick stride',layout=ipyw.Layout(width='120px'))
    xmintick    = ipyw.IntText(value=5,layout=ipyw.Layout(width='60px'))
    xmintk_unit = ipyw.Label(value='years')

    # Y-axis buffer
    txt_buff = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis buffer</a></h3>')
    maxb_desc = ipyw.Label(value='Buffer above y-axis',layout=ipyw.Layout(width='120px'))
    maxbuff   = ipyw.FloatText(value=1.,layout=ipyw.Layout(width='60px'))
    maxb_unit = ipyw.Label(value='inches')
    minb_desc = ipyw.Label(value='Buffer below y-axis',layout=ipyw.Layout(width='120px'))
    minbuff   = ipyw.FloatText(value=1.,layout=ipyw.Layout(width='60px'))
    minb_unit = ipyw.Label(value='inches')

    # Map
    txt_map   = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_map} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Map of stations</a></h3>')
    text_map  = ipyw.Label(value='Include map?',layout=ipyw.Layout(width='100px'))
    incl_map  = ipyw.Dropdown(options=['True (left)','True (right)','False'],value='True (left)',
                              layout=ipyw.Layout(width='90px'))
    text_lyr  = ipyw.Label(value='Image tile',layout=ipyw.Layout(width='100px'))
    img_tile  = ipyw.Dropdown(options=['QuadtreeTiles','GoogleTiles','OpenStreetMap','grey'],
                              value='QuadtreeTiles',layout=ipyw.Layout(width='80px'))
    text_lbl = ipyw.Label(value='Label distance',layout=ipyw.Layout(width='100px'))
    lbl_buff = ipyw.Dropdown(options=[('10%',0.1),('20%',0.2),('30%',0.3),('40%',0.4),('50%',0.5),
                                       ('60%',0.6),('70%',0.7),('80%',0.8),('90%',0.9),('None',1.5)],
                              value=0.3,layout=ipyw.Layout(width='90px'))
    text_ext = ipyw.Label(value='Lat/lon buffer',layout=ipyw.Layout(width='100px'))
    ext_buff = ipyw.Dropdown(options=[('0.1°',0.1),('0.25°',0.25),('0.5°',0.5),('1°',1.),('5°',5.)],
                              value=0.5,layout=ipyw.Layout(width='90px'))

    #----------------------------------------------------------------------------------------------
    # Layout for dropdowns
    #----------------------------------------------------------------------------------------------

    ui = ipyw.VBox([
                    # FIRST ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Specify months to include
                                 ipyw.HBox([txt_monthi],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_mon1,month1]),
                                                       ipyw.HBox([text_mon2,month2])])],
                                           layout=ipyw.Layout(justify_content='center'))]),
                               ipyw.VBox([
                                 # Method
                                 ipyw.HBox([txt_meth],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_meth,method]),
                                 ipyw.HBox([text_tl,incl_tl]),
                                 ipyw.HBox([text_yr,tl_syr,tl_eyr])]),
                               ipyw.VBox([
                                 # Data quality standards
                                 ipyw.HBox([txt_dq],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_ndays,num_days]),
                                                       ipyw.HBox([text_nmons,num_mons]),
                                                       ipyw.HBox([text_stns,num_stns])])])])],
                               layout=ipyw.Layout(justify_content='space-around')),
                    # SECOND ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Tick stride
                                 ipyw.HBox([txt_tick],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([ymajtk_desc,ymajtick,ymajtk_unit]),
                                                       ipyw.HBox([ymintk_desc,ymintick,ymintk_unit]),
                                                       ipyw.HBox([xmajtk_desc,xmajtick,xmajtk_unit]),
                                                       ipyw.HBox([xmintk_desc,xmintick,xmintk_unit])])],
                                            layout=ipyw.Layout(flex='1 1 auto',
                                                               justify_content='space-between'))]),
                               ipyw.VBox([
                                 # Y-axis buffer
                                 ipyw.HBox([txt_buff],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([maxb_desc,maxbuff,maxb_unit]),
                                                       ipyw.HBox([minb_desc,minbuff,minb_unit])])],
                                           layout=ipyw.Layout(flex='1 1 auto',
                                                              justify_content='center'))]),
                               ipyw.VBox([
                                  # Map 
                                 ipyw.HBox([txt_map],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_map,incl_map]),
                                                       ipyw.HBox([text_lyr,img_tile]),
                                                       ipyw.HBox([text_lbl,lbl_buff]),
                                                       ipyw.HBox([text_ext,ext_buff])])])])],
                               layout=ipyw.Layout(justify_content='space-around'))
                     ])

    #----------------------------------------------------------------------------------------------
    # Display interactive plot
    #----------------------------------------------------------------------------------------------

    # Update dropdown values to reflect magnitude change between methods
    def update_dropdowns(change):
        if change.new in ['rx1day-max', 'rx1day-mean']:
            ymajtick.value,ymintick.value,maxbuff.value,minbuff.value = 1., 0.5, 1., 1.
        elif change.new in ['alldays-mean', 'snowdays-mean']:
            print('Hold tight while the widget readjusts itself...')
            ymajtick.value,ymintick.value,maxbuff.value,minbuff.value = 0.05, 0.01, 0.05, 0.05
    method.observe(update_dropdowns, names='value')
    
    # Define function with only interactive components that runs widget.plot function
    def plot_fig(month1,month2,num_days,num_mons,num_stns,method,incl_tl,tl_syr,tl_eyr,minbuff,
                 maxbuff,ymajtick,ymintick,xmajtick,xmintick,incl_map,img_tile,lbl_buff,ext_buff):
        
        fig,ts = plots.timeseries_snow_plot(
                        var=var,meta=meta,location_name=location_name,nlat=float(nlat),
                        slat=float(slat),wlon=float(wlon),elon=float(elon),month1=month1,
                        month2=month2,num_days=num_days,num_mons=num_mons,num_stns=num_stns,
                        method=method,incl_tl=incl_tl,tl_syr=tl_syr,tl_eyr=tl_eyr,minbuff=minbuff,
                        maxbuff=maxbuff,ymajtick=ymajtick,ymintick=ymintick,xmajtick=xmajtick,
                        xmintick=xmintick,incl_map=incl_map,img_tile=img_tile,lbl_buff=lbl_buff,
                        ext_buff=ext_buff)

        # Create a button widget to download the figure as PDF
        pdf_output = ipyw.Output()
        def download_pdf(button):
           pdf_filename = 'figure.pdf'
           pdf_opts(fig=fig,pdf_output=pdf_output,pdf_filename=pdf_filename)
        pdf_but = ipyw.Button(description='Download Figure', layout={'width': '140px'})
        pdf_but.on_click(download_pdf)

        # Create a button widget to download data as Excel
        xcl_output = ipyw.Output()
        def download_xcl(button):
            with xcl_output: print('Download initiated. Can take more than a minute if over 100 stations '+
                                   'are queried. Do not click download button again.')
            xcl_filename = 'data.xlsx'
            with pd.ExcelWriter(xcl_filename) as w:
                var.assign(**{var.columns[0]: var.iloc[:,0].astype(str)}
                           ).to_excel(w,sheet_name='stations',index=False)
                meta.to_excel(w,sheet_name='metadata',index=False)
                ts.rename(columns={'Value':method}).to_excel(w,sheet_name='timeseries',index=False)
            xcl_opts(xcl_filename=xcl_filename,xcl_output=xcl_output)
        xcl_but = ipyw.Button(description='Download Data',layout=ipyw.Layout(width='140px'))
        xcl_but.on_click(download_xcl)

        # Display download buttons together
        button_box = ipyw.VBox([ipyw.HBox([pdf_but,xcl_but],layout=ipyw.Layout(justify_content='center'))])
        display(button_box,pdf_output,xcl_output)

    # Interactive output with only interactive components in dictionary
    out = ipyw.interactive_output(plot_fig,{'month1':month1,'month2':month2,'num_days':num_days,
                                            'num_mons':num_mons,'num_stns':num_stns,'method':method,
                                            'incl_tl':incl_tl,'tl_syr':tl_syr,'tl_eyr':tl_eyr,
                                            'minbuff':minbuff,'maxbuff':maxbuff,'ymajtick':ymajtick,
                                            'ymintick':ymintick,'xmajtick':xmajtick,'xmintick':xmintick,
                                            'incl_map':incl_map,'img_tile':img_tile,'lbl_buff':lbl_buff,
                                            'ext_buff':ext_buff})
    display(ui,out)


In [4]:
#-----------------
# Run function    
#-----------------

timeseries_snow_widget(var=var,meta=meta,location_name=location_name,nlat=nlat,slat=slat,wlon=wlon,elon=elon)


Output()